# Getting started

- First run below command to install Xla library that inferfaces b/w Pytorch and the TPU
- Also we need awscli to connect our database

In [1]:
src_location = 'drive/MyDrive/Sharing_storage1/custom.zip'
base_location ='compressed_zip'
!cp {src_location} {base_location} 
print('copying over the file')

copying over the file


In [2]:
!unzip 'compressed_zip'
# !rm 'compressed_zip' 

print('we are ready to go')

Streaming output truncated to the last 5000 lines.
  inflating: custom/test/Jeep_Wrangler_2018/Jeep_Wrangler_2018_187.jpg  
  inflating: custom/test/Jeep_Wrangler_2018/Jeep_Wrangler_2018_191.jpg  
  inflating: custom/test/Jeep_Wrangler_2018/Jeep_Wrangler_2018_193.jpg  
  inflating: custom/test/Jeep_Wrangler_2018/Jeep_Wrangler_2018_194.jpg  
  inflating: custom/test/Jeep_Wrangler_2018/Jeep_Wrangler_2018_196.jpg  
  inflating: custom/test/Jeep_Wrangler_2018/Jeep_Wrangler_2018_197.jpg  
   creating: custom/test/Merces-Benz_C_Sedan_2018/
  inflating: custom/test/Merces-Benz_C_Sedan_2018/Merces-Benz_C_Sedan_2018_000.jpg  
  inflating: custom/test/Merces-Benz_C_Sedan_2018/Merces-Benz_C_Sedan_2018_001.jpg  
  inflating: custom/test/Merces-Benz_C_Sedan_2018/Merces-Benz_C_Sedan_2018_002.jpg  
  inflating: custom/test/Merces-Benz_C_Sedan_2018/Merces-Benz_C_Sedan_2018_009.jpg  
  inflating: custom/test/Merces-Benz_C_Sedan_2018/Merces-Benz_C_Sedan_2018_011.jpg  
  inflating: custom/test/Merces-Ben

In [3]:
!pip install d2l==0.15.1 -q
!pip install -U mxnet-cu101==1.7.0 -q
!pip install pytorch-lightning -q

print('done setting things up')


     |████████████████████████████████| 61kB 3.2MB/s 
     |███████████████████████████████▌| 834.1MB 1.3MB/s eta 0:00:10tcmalloc: large alloc 1147494400 bytes == 0x653aa000 @  0x7f2ffda70615 0x591e47 0x4cc179 0x4cc2db 0x50a1cc 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50cc96 0x58e683 0x50c127 0x58e683 0x50c127 0x58e683 0x50c127 0x58e683 0x50c127 0x5095c8 0x50a2fd 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50beb4 0x5095c8 0x50a2fd
     |████████████████████████████████| 846.0MB 22kB/s 
     |████████████████████████████████| 563kB 4.3MB/s 
     |████████████████████████████████| 92kB 10.2MB/s 
     |████████████████████████████████| 829kB 52.4MB/s 
     |████████████████████████████████| 276kB 44.7MB/s 
done setting things up


- Check how many GPU you have

In [4]:
import torch
import numpy as np
import random
from torch import nn
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F
import pytorch_lightning as pl
import os
from PIL import Image
import logging
from datetime import datetime


number_gpu = torch.cuda.device_count()
print('the number of available gpu at the moment is:{}'.format(number_gpu))

!nvidia-smi

the number of available gpu at the moment is:1
Sun Dec  6 09:31:55 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    26W / 250W |     10MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                         

# ResNet Module

In [5]:
from  torch.utils.model_zoo import load_url as load_state_dict_from_url


__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
           'resnet152', 'resnext50_32x4d', 'resnext101_32x8d']


model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}


def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x1 = self.maxpool(x)

        x2 = self.layer1(x1)
        x3 = self.layer2(x2)
        x4 = self.layer3(x3)
        x5 = self.layer4(x4)

        x = self.avgpool(x5)
        x = x.reshape(x.size(0), -1)
        x = self.fc(x)

        return x1, x2, x3, x4, x5


def _resnet(arch, inplanes, planes, pretrained, progress, **kwargs):
    model = ResNet(inplanes, planes, **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model


def resnet18(pretrained=False, progress=True, **kwargs):
    """Constructs a ResNet-18 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet18', BasicBlock, [2, 2, 2, 2], pretrained, progress,
                   **kwargs)


def resnet34(pretrained=False, progress=True, **kwargs):
    """Constructs a ResNet-34 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet34', BasicBlock, [3, 4, 6, 3], pretrained, progress,
                   **kwargs)


def resnet50(pretrained=False, progress=True, **kwargs):
    """Constructs a ResNet-50 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet50', Bottleneck, [3, 4, 6, 3], pretrained, progress,
                   **kwargs)


def resnet101(pretrained=False, progress=True, **kwargs):
    """Constructs a ResNet-101 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet101', Bottleneck, [3, 4, 23, 3], pretrained, progress,
                   **kwargs)


def resnet152(pretrained=False, progress=True, **kwargs):
    """Constructs a ResNet-152 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet152', Bottleneck, [3, 8, 36, 3], pretrained, progress,
                   **kwargs)


def resnext50_32x4d(**kwargs):
    kwargs['groups'] = 32
    kwargs['width_per_group'] = 4
    return _resnet('resnext50_32x4d', Bottleneck, [3, 4, 6, 3],
                   pretrained=False, progress=True, **kwargs)


def resnext101_32x8d(**kwargs):
    kwargs['groups'] = 32
    kwargs['width_per_group'] = 8
    return _resnet('resnext101_32x8d', Bottleneck, [3, 4, 23, 3],
                   pretrained=False, progress=True, **kwargs)
    
print('works fine')

works fine


# Utils


In [6]:
import numpy as np
import random
import torch
import torchvision
from torch.autograd import Variable
from torchvision import transforms, models

#! Not using this
def cosine_anneal_schedule(t, nb_epoch, lr):
    # t - 1 is used when t has 1-based indexing.
    cos_inner = np.pi * (t % (nb_epoch))
    cos_inner /= (nb_epoch)
    cos_out = np.cos(cos_inner) + 1

    return float(lr / 2 * cos_out)


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


def jigsaw_generator(images, n):
    l = []
    for a in range(n):
        for b in range(n):
            l.append([a, b])
    block_size = 448 // n
    rounds = n ** 2
    random.shuffle(l)
    jigsaws = images.clone()
    for i in range(rounds):
        x, y = l[i]
        temp = jigsaws[..., 0:block_size, 0:block_size].clone()
        jigsaws[..., 0:block_size, 0:block_size] = jigsaws[..., x * block_size:(x + 1) * block_size,
                                                           y * block_size:(y + 1) * block_size].clone()
        jigsaws[..., x * block_size:(x + 1) * block_size,
                y * block_size:(y + 1) * block_size] = temp

    return jigsaws

# This probably doesn't have to be a lightning module
class BasicConv(pl.LightningModule):
    def __init__(self, in_planes, out_planes, kernel_size, stride=1, padding=0, dilation=1, groups=1, relu=True, bn=True, bias=False):
        super(BasicConv, self).__init__()
        self.out_channels = out_planes
        self.conv = nn.Conv2d(in_planes, out_planes, kernel_size=kernel_size,
                              stride=stride, padding=padding, dilation=dilation, groups=groups, bias=bias)
        self.bn = nn.BatchNorm2d(out_planes, eps=1e-5,
                                 momentum=0.01, affine=True) if bn else None
        self.relu = nn.ReLU() if relu else None

    def forward(self, x):
        x = self.conv(x)
        if self.bn is not None:
            x = self.bn(x)
        if self.relu is not None:
            x = self.relu(x)
        return x

print('works fine too')


works fine too


# Loss Function Tweek

In [7]:
class SmoothCrossEntropyLoss(_WeightedLoss):

    """
    This applies simple label smoothing to the cross entropy with strength parameter of 0.2
    """

    def __init__(self, weight=None, reduction='mean', smoothing=0.2):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth_one_hot(targets:torch.Tensor, n_classes:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = torch.empty(size=(targets.size(0), n_classes),
                    device=targets.device) \
                .fill_(smoothing /(n_classes-1)) \
                .scatter_(1, targets.data.unsqueeze(1), 1.-smoothing)
        return targets

    def forward(self, inputs, targets):
        targets = SmoothCrossEntropyLoss._smooth_one_hot(targets, inputs.size(-1),
            self.smoothing)
        lsm = F.log_softmax(inputs, -1)

        if self.weight is not None:
            lsm = lsm * self.weight.unsqueeze(0)

        loss = -(targets * lsm).sum(-1)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

class LargeMarginInSoftmaxLoss(nn.CrossEntropyLoss):
    """
    This combines the Softmax Cross-Entropy Loss (nn.CrossEntropyLoss) and the large-margin inducing
    """
    def __init__(self, reg_lambda=0.1, deg_logit=None, 
                weight=None, size_average=None, ignore_index=-100, reduce=None, reduction='mean'):
        super(LargeMarginInSoftmaxLoss, self).__init__(weight=weight, size_average=size_average, 
                                ignore_index=ignore_index, reduce=reduce, reduction=reduction)
        self.reg_lambda = reg_lambda
        self.deg_logit = deg_logit

    def forward(self, input, target):
        N = input.size(0) # number of samples
        C = input.size(1) # number of classes
        Mask = torch.zeros_like(input, requires_grad=False)
        Mask[range(N),target] = 1

        if self.deg_logit is not None:
            input = input - self.deg_logit * Mask
        
        loss = F.cross_entropy(input, target, weight=self.weight,
                               ignore_index=self.ignore_index, reduction=self.reduction)

        X = input - 1.e6 * Mask # [N x C], excluding the target class
        reg = 0.5 * ((F.softmax(X, dim=1) - 1.0/(C-1)) * F.log_softmax(X, dim=1) * (1.0-Mask)).sum(dim=1)
        if self.reduction == 'sum':
            reg = reg.sum()
        elif self.reduction == 'mean':
            reg = reg.mean()
        elif self.reduction == 'none':
            reg = reg

        return loss + self.reg_lambda * reg


class ComplementEntropy(nn.Module):
    '''Compute the complement entropy of complement classes.'''
    def __init__(self, num_classes=200):
        super(ComplementEntropy, self).__init__()
        self.classes = num_classes
        self.batch_size = None

    def forward(self, y_hat, y):
        self.batch_size = len(y)
        y_hat = F.softmax(y_hat, dim=1)
        Yg = torch.gather(y_hat, 1, torch.unsqueeze(y, 1))
        Yg_ = (1 - Yg) + 1e-7
        Px = y_hat / Yg_.view(len(y_hat), 1)
        Px_log = torch.log(Px + 1e-10)
        y_zerohot = torch.ones(self.batch_size, self.classes).scatter_\
            (1, y.view(self.batch_size, 1).data.cpu(), 0)
        output = Px * Px_log * y_zerohot.cuda()
        entropy = torch.sum(output)
        entropy /= float(self.batch_size)
        entropy /= float(self.classes)
        return entropy


class ComplementCrossEntropy(nn.Module):
    def __init__(self, num_classes=200, gamma=5):
        super(ComplementCrossEntropy, self).__init__()
        self.gamma = gamma
        self.cross_entropy = nn.CrossEntropyLoss()
        self.complement_entropy = ComplementEntropy(num_classes)

    def forward(self, y_hat, y):
        l1 = self.cross_entropy(y_hat, y)
        l2 = self.complement_entropy(y_hat, y)
        return l1 + self.gamma * l2

print('all good')

all good


# Main Model

In [8]:
import torch.optim as optim
from pytorch_lightning.metrics.functional.classification import accuracy


# This is model is written considering L = 5 and S = 3 (so S + 1 steps)
class PMG(pl.LightningModule):

    def __init__(self, model, feature_size, lr, loss, classes_num, reg, batch_size=8, num_workers=6, root='car_data'):
        super(PMG, self).__init__()

        self.features = model  
        self.max1 = nn.MaxPool2d(kernel_size=56, stride=56)
        self.max2 = nn.MaxPool2d(kernel_size=28, stride=28)
        self.max3 = nn.MaxPool2d(kernel_size=14, stride=14)
        self.num_ftrs = 2048 * 1 * 1
        self.elu = nn.ELU(inplace=True)
        
        self.root = root
        self.reg = reg
        self.loss = loss
        self.lr = lr
        self.batch_size = batch_size
        self.num_workers = num_workers

        """
        ----------------------------------------
        Refer to graph on page 6.
        This is Conv block L-2 and classfier L-2
        ----------------------------------------
        """
        self.conv_block1 = nn.Sequential(
            BasicConv(self.num_ftrs//4, feature_size, kernel_size=1,
                      stride=1, padding=0, relu=True),
            BasicConv(feature_size, self.num_ftrs//2,
                      kernel_size=3, stride=1, padding=1, relu=True)
        )

        self.classifier1 = nn.Sequential(
            nn.BatchNorm1d(self.num_ftrs//2),
            nn.Linear(self.num_ftrs//2, feature_size),
            nn.BatchNorm1d(feature_size),
            nn.ELU(inplace=True),
            nn.Linear(feature_size, classes_num),
        )

        """
        -----------------------------------------
        This is Conv block L-1 and classfier L-1
        -----------------------------------------
        """
        self.conv_block2 = nn.Sequential(
            BasicConv(self.num_ftrs//2, feature_size, kernel_size=1,
                      stride=1, padding=0, relu=True),
            BasicConv(feature_size, self.num_ftrs//2,
                      kernel_size=3, stride=1, padding=1, relu=True)
        )

        self.classifier2 = nn.Sequential(
            nn.BatchNorm1d(self.num_ftrs//2),
            nn.Linear(self.num_ftrs//2, feature_size),
            nn.BatchNorm1d(feature_size),
            nn.ELU(inplace=True),
            nn.Linear(feature_size, classes_num),
        )

        """
        ----------------------------------------
        This is Conv block L and classfier L
        ----------------------------------------
        """
        self.conv_block3 = nn.Sequential(
            BasicConv(self.num_ftrs, feature_size, kernel_size=1,
                      stride=1, padding=0, relu=True),
            BasicConv(feature_size, self.num_ftrs//2,
                      kernel_size=3, stride=1, padding=1, relu=True)
        )
        self.classifier3 = nn.Sequential(
            nn.BatchNorm1d(self.num_ftrs//2),
            nn.Linear(self.num_ftrs//2, feature_size),
            nn.BatchNorm1d(feature_size),
            nn.ELU(inplace=True),
            nn.Linear(feature_size, classes_num),
        )

        """
        ----------------------------------------
        Refer to graph on page 6 and 7.
        This is the classifier concat
        ----------------------------------------
        """
        self.classifier_concat = nn.Sequential(
            nn.BatchNorm1d(1024 * 3),
            nn.Linear(1024 * 3, feature_size),
            nn.BatchNorm1d(feature_size),
            nn.ELU(inplace=True),
            nn.Linear(feature_size, classes_num),
        )

    def forward(self, x):

        xf1, xf2, xf3, xf4, xf5 = self.features(x)

        xl1 = self.conv_block1(xf3)
        xl2 = self.conv_block2(xf4)
        xl3 = self.conv_block3(xf5)

        xl1 = self.max1(xl1)
        xl1 = xl1.view(xl1.size(0), -1)
        xc1 = self.classifier1(xl1)

        xl2 = self.max2(xl2)
        xl2 = xl2.view(xl2.size(0), -1)
        xc2 = self.classifier2(xl2)

        xl3 = self.max3(xl3)
        xl3 = xl3.view(xl3.size(0), -1)
        xc3 = self.classifier3(xl3)

        """
        xcl, xc2, xc3 will be used to calculate the loss.
        x_concat is just concat of layer features (1,2,3) 
        """
        x_concat = torch.cat((xl1, xl2, xl3), -1)
        x_concat = self.classifier_concat(x_concat)
        return xc1, xc2, xc3, x_concat

    # lightning will add optimizer inside the model
    def configure_optimizers(self):

        optimizer = optim.SGD([
            {'params': self.classifier_concat.parameters(), 'lr': self.lr},
            {'params': self.conv_block1.parameters(), 'lr': self.lr},
            {'params': self.classifier1.parameters(), 'lr': self.lr},
            {'params': self.conv_block2.parameters(), 'lr': self.lr},
            {'params': self.classifier2.parameters(), 'lr': self.lr},
            {'params': self.conv_block3.parameters(), 'lr': self.lr},
            {'params': self.classifier3.parameters(), 'lr': self.lr},
            {'params': self.features.parameters(), 'lr': self.lr/10}
        ],
            momentum=0.9, weight_decay=5e-4)
        
        # Learning rate optimizer options.
        cosineAnneal = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, verbose=True)
        #plateau = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, verbose=True) # you need to specify what you are monitoring. 

        step_lr = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1)

        warm_restart = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, verbose=True)
        
        return {'optimizer': optimizer, 'lr_scheduler': cosineAnneal}

    def training_step(self, batch, batch_idx):
      
        inputs, targets = batch
        loss_function = self.loss

        # step 1 (start from fine-grained jigsaw n=8)
        inputs1 = jigsaw_generator(inputs, 8)
        output_1, _, _, _ = self(inputs1) 
        loss1 = loss_function(output_1, targets) * 1 

        # step 2
        inputs2 = jigsaw_generator(inputs, 4)
        _, output_2, _, _ = self(inputs2)
        loss2 = loss_function(output_2, targets) * 1  

        # step 3
        inputs3 = jigsaw_generator(inputs, 2)
        _, _, output_3, _ = self(inputs3)
        loss3 = loss_function(output_3, targets) * 1

        # step 4 whole image, and vanilla loss. 
        _, _, _, output_concat = self(inputs)

        if self.reg == None:
            concat_loss = loss_function(output_concat, targets) * 2 
            train_loss = loss1 + loss2 + loss3 + concat_loss

        if self.reg == 'large_margin':
            pass

        elif self.reg == 'jacobian':
            pass

        # accuracy 
        _, predicted = torch.max(output_concat.data, 1)
        train_acc = accuracy(predicted, targets)

        metrics = {'loss': train_loss, 'accuracy': train_acc}

        self.log('accuracy', train_acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        
        return metrics

    # * Not entirely the same as the training step. No jigsaw puzzle here.
    def validation_step(self, batch, batch_idx):

        inputs, targets = batch

        loss_function = self.loss
        
        #TODO: JOKER LOSS is probably not needed for validation
        output_1, output_2, output_3, output_concat = self(inputs)
        outputs_com = output_1 + output_2 + output_3 + output_3 + output_concat

        val_loss = loss_function(output_concat, targets)

        """
        There is the individual accuracy, and combined accuracy
        """
        _, predicted = torch.max(output_concat.data, 1)
        _, predicted_com = torch.max(outputs_com.data, 1)

        valid_acc = accuracy(predicted, targets)
        valid_acc_en = accuracy(predicted_com, targets)

        metrics = {'val_loss':  val_loss, 'val_acc': valid_acc, 'val_acc_en': valid_acc_en}

        self.log('val_acc', valid_acc, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log('val_acc_en', valid_acc_en, on_step=False, on_epoch=True, prog_bar=True, logger=True)


        return metrics

    
    def test_step(self, batch, batch_idx):

        metrics = self.validation_step(batch, batch_idx)
        metrics = {'test_acc' : metrics['val_acc_en'], 'test_loss': metrics['val_loss']}
        self.log_dict(metrics)


    def train_dataloader(self):

        transform_train = transforms.Compose([
            transforms.Resize((550, 550)),
            transforms.RandomCrop(448, padding=8),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ])

        trainset = torchvision.datasets.ImageFolder(
            root= self.root+'/train', transform=transform_train)
        trainloader = torch.utils.data.DataLoader(
            trainset, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers, pin_memory=True)

        return trainloader

    def val_dataloader(self):

        transform_test = transforms.Compose([
            transforms.Resize((550, 550)),
            transforms.CenterCrop(448),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ])

        # ? Shuffle was True in the original implementation. This is most likely not the best practice.
        testset = torchvision.datasets.ImageFolder(root=self.root+'/test',
                                                   transform=transform_test)
        testloader = torch.utils.data.DataLoader(
            testset, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers, pin_memory=True)

        return testloader

print('model fine too')

model fine too


# Config

In [9]:
BATCH_SIZE = 2 # this will be automatically tuned and increased to best fit your machine.
NUM_WORKERS = 6
MODEL_BASE = 'resnet50_pmg'
LEARNING_RATE = 0.002
CLASSES = 100
LOSS= 'ce_vanilla' #options: ce_vanilla, ce_label_smooth, complement, large_margin
EPOCH = 300

# Training
- Try the Neptune Logger here

In [10]:
from pytorch_lightning.callbacks import ProgressBar, GradientAccumulationScheduler, ModelCheckpoint, EarlyStopping
from pytorch_lightning.tuner.tuning import Tuner
from pytorch_lightning.loggers import TensorBoardLogger, CSVLogger


def load_model(model_name, loss, learning_rate, batch_size, num_workers, regularizer, pretrain=True):
    print('==> Building model..')
    if model_name == 'resnet50_pmg':
        net = resnet50(pretrained=pretrain)
        net = PMG(net, loss=loss, feature_size = 512, classes_num = CLASSES, batch_size=batch_size,
                  num_workers=num_workers, lr = learning_rate, reg=regularizer, root='custom') ## this should work right?

    return net

"""
----------------
Main Area
-----------------
"""

torch.manual_seed(0)
np.random.seed(0)
# torch.set_deterministic(True)

model_name = MODEL_BASE
time = datetime.now().strftime('%Y%m%d_%H%M%S')
save_dir = 'weights/{}/{}'.format(time,LOSS)
reg_type = None

if LOSS == 'ce_vanilla':
    print('==> The loss function is set as: ', LOSS)
    loss = nn.CrossEntropyLoss()

elif LOSS == 'ce_label_smooth':
    print('==> The loss function is set as: ', LOSS)
    loss = SmoothCrossEntropyLoss()

elif LOSS == 'large_margin':
    loss = nn.CrossEntropyLoss()
    reg_type = 'large_margin'

elif LOSS == 'complement':
    loss = ComplementCrossEntropy()

else:
    print('====> The LOSS IS NOT SET PROPERLY')
        
model = load_model(model_name, loss, LEARNING_RATE, BATCH_SIZE, NUM_WORKERS, pretrain=True, regularizer=reg_type)

print('The model has {:,} trainable parameters'.format(
    count_parameters(model)))


"""
------------------------------------
Intialize the trainier from the model and the callbacks
# Tensorboard logging: tensorboard --logdir lightning_logs
------------------------------------
"""

bar = ProgressBar()
# early_stopping = EarlyStopping('val_acc_en', patience=15)

# Two validation metrics. Let's have two different saver. 
ckpt_en = ModelCheckpoint(
    dirpath=save_dir, monitor='val_acc_en', mode = 'auto', save_last=True, filename='{epoch:02d}-{val_acc_en:.4f}')

ckpt_reg = ModelCheckpoint(
    dirpath=save_dir, monitor='val_acc', mode = 'auto', save_last=True, filename='{epoch:02d}-{val_acc:.4f}')


csv_logger = CSVLogger('csv_log/{}'.format(time), name = '{}_model'.format(LOSS))
tensorboard_logger = TensorBoardLogger('tb_log/{}'.format(time), name='{}_model'.format(LOSS))

# use resume_from_checkpoint
# resume_point = 'weights/20201201_180420/ce_label_smooth/last.ckpt'

trainer = pl.Trainer(auto_scale_batch_size='power', callbacks=[bar, ckpt_en, ckpt_reg],
                      max_epochs=EPOCH, gpus=1, precision=16, logger = [tensorboard_logger, csv_logger])

print('==> Starting the training process now...')

trainer.tune(model) 
trainer.fit(model)


==> The loss function is set as:  ce_vanilla
==> Building model..


Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.


The model has 44,926,392 trainable parameters
==> Starting the training process now...
Adjusting learning rate of group 0 to 2.0000e-03.
Adjusting learning rate of group 1 to 2.0000e-03.
Adjusting learning rate of group 2 to 2.0000e-03.
Adjusting learning rate of group 3 to 2.0000e-03.
Adjusting learning rate of group 4 to 2.0000e-03.
Adjusting learning rate of group 5 to 2.0000e-03.
Adjusting learning rate of group 6 to 2.0000e-03.
Adjusting learning rate of group 7 to 2.0000e-04.
Epoch     0: adjusting learning rate of group 0 to 2.0000e-03.
Epoch     0: adjusting learning rate of group 1 to 2.0000e-03.
Epoch     0: adjusting learning rate of group 2 to 2.0000e-03.
Epoch     0: adjusting learning rate of group 3 to 2.0000e-03.
Epoch     0: adjusting learning rate of group 4 to 2.0000e-03.
Epoch     0: adjusting learning rate of group 5 to 2.0000e-03.
Epoch     0: adjusting learning rate of group 6 to 2.0000e-03.
Epoch     0: adjusting learning rate of group 7 to 2.0000e-04.


Batch size 2 succeeded, trying batch size 4


...

Adjusting learning rate of group 0 to 1.8090e-03.
Adjusting learning rate of group 1 to 1.8090e-03.
Adjusting learning rate of group 2 to 1.8090e-03.
Adjusting learning rate of group 3 to 1.8090e-03.
Adjusting learning rate of group 4 to 1.8090e-03.
Adjusting learning rate of group 5 to 1.8090e-03.
Adjusting learning rate of group 6 to 1.8090e-03.
Adjusting learning rate of group 7 to 1.8090e-04.


Adjusting learning rate of group 0 to 1.9511e-03.
Adjusting learning rate of group 1 to 1.9511e-03.
Adjusting learning rate of group 2 to 1.9511e-03.
Adjusting learning rate of group 3 to 1.9511e-03.
Adjusting learning rate of group 4 to 1.9511e-03.
Adjusting learning rate of group 5 to 1.9511e-03.
Adjusting learning rate of group 6 to 1.9511e-03.
Adjusting learning rate of group 7 to 1.9511e-04.


Adjusting learning rate of group 0 to 2.0000e-03.
Adjusting learning rate of group 1 to 2.0000e-03.
Adjusting learning rate of group 2 to 2.0000e-03.
Adjusting learning rate of group 3 to 2.0000e-03.
Adjusting learning rate of group 4 to 2.0000e-03.
Adjusting learning rate of group 5 to 2.0000e-03.
Adjusting learning rate of group 6 to 2.0000e-03.
Adjusting learning rate of group 7 to 2.0000e-04.


/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


1

In [12]:
best_val_acc = 'weights/20201206_093321/ce_vanilla/epoch=46-val_acc=0.9573.ckpt'
best_en = 'weights/20201206_093321/ce_vanilla/epoch=46-val_acc_en=0.9586.ckpt'

src_val_acc = 'drive/MyDrive/Sharing_storage1/weights_backup/custom/baseline/val_acc_0.9573.ckpt'
src_val_acc_en = 'drive/MyDrive/Sharing_storage1/weights_backup/custom/baseline/val_acc_en_0.9586.ckpt'


!cp {best_val_acc} {src_val_acc}
!cp {best_en} {src_val_acc_en}

print('copying completed')

copying completed
